<a href="https://colab.research.google.com/github/avk1943/MS-Project/blob/main/Data_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import h5py

In [ ]:
dir_name = r'/content/drive/MyDrive/Extract_Driving_Events'

files_in_dir = [os.path.join(path, file) for path, dirs,
                files in os.walk(dir_name) for file in files]
print(files_in_dir)

['/content/drive/MyDrive/Extract_Driving_Events/s01_060227n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_1n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_2n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s02_050921m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s02_051115m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s04_051130m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_051120m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_060308n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_061019m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_061101n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s06_051119m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s09_060313n_event_data.mat']


In [ ]:
m_X = np.empty((0, 30, 1000), float)
m_y = np.empty((0,), int)


for file_name in files_in_dir:
  f = h5py.File(file_name,'r')
  print(file_name)
  data = f.get('driver_baseline/eegData')
  data = np.array(data) # For converting to a NumPy array

  data = f.get('driver_baseline/eegData')
  data = np.array(data)
  baseline = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    baseline = np.concatenate((baseline,temp))
  #print(baseline.shape)

  data = f.get('driver_baseline/eegTime')
  data = np.array(data)
  baselineT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    baselineT = np.concatenate((baselineT,temp))
  #print(baselineT.shape)

  data = f.get('driver_correction/eegData')
  data = np.array(data)
  correction = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    correction = np.concatenate((correction,temp))
  #print(correction.shape)

  data = f.get('driver_correction/eegTime')
  data = np.array(data)
  correctionT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    correctionT = np.concatenate((correctionT,temp))
  #print(correctionT.shape)

  data = f.get('driver_disturbance/eegData')
  data = np.array(data)
  disturbance = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    disturbance = np.concatenate((disturbance,temp))
  #print(disturbance.shape)

  data = f.get('driver_disturbance/eegTime')
  data = np.array(data)
  disturbanceT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    disturbanceT = np.concatenate((disturbanceT,temp))
  #print(disturbanceT.shape)


  X = np.concatenate((baseline,correction,disturbance))
  #print("X success")
  #print(X.shape)
  Xt = np.concatenate((baselineT,correctionT,disturbanceT))
  #print("Xt success")
  #print(Xt.shape)
  y = [0 for i in range(len(baseline))] + [1 for i in range(len(correction))] + [2 for i in range(len(disturbance))]
  y = np.array(y)
  #print("y success")
  #print(y.shape)

  Xnew = np.ndarray((len(X)//1000+1,30,1000))
  ynew = []
  counter = 0
  size = 0
  i = 0
  while i < len(Xt)-1000:
    size = Xt[i+1]-Xt[i]

    if np.isclose(Xt[i+1000]-Xt[i],size*1000) :
      Xnew[counter] = np.transpose(X[i:i+1000])
      ynew.append(y[i])
      i = i+1000
      counter = counter+1
    else:
      i=i+1

  X = Xnew[:counter]
  y = ynew
  print(len(y))
  y = np.asarray(y)
  print("New data:")
  print("X shape:")
  print(X.shape)
  print("y shape:")
  print(y.shape)

  m_X = np.append(m_X, X, axis=0)
  print("M_x shape:")
  print(m_X.shape)
  m_y = np.append(m_y, y, axis=0)
  print("M_y shape:")
  print(m_y.shape)


/content/drive/MyDrive/Extract_Driving_Events/s01_060227n_event_data.mat
1403
New data:
X shape:
(1403, 30, 1000)
y shape:
(1403,)
M_x shape:
(1403, 30, 1000)
M_y shape:
(1403,)
/content/drive/MyDrive/Extract_Driving_Events/s01_060926_1n_event_data.mat
1717
New data:
X shape:
(1717, 30, 1000)
y shape:
(1717,)
M_x shape:
(3120, 30, 1000)
M_y shape:
(3120,)
/content/drive/MyDrive/Extract_Driving_Events/s01_060926_2n_event_data.mat
1456
New data:
X shape:
(1456, 30, 1000)
y shape:
(1456,)
M_x shape:
(4576, 30, 1000)
M_y shape:
(4576,)
/content/drive/MyDrive/Extract_Driving_Events/s02_050921m_event_data.mat
1399
New data:
X shape:
(1399, 30, 1000)
y shape:
(1399,)
M_x shape:
(5975, 30, 1000)
M_y shape:
(5975,)
/content/drive/MyDrive/Extract_Driving_Events/s02_051115m_event_data.mat
1427
New data:
X shape:
(1427, 30, 1000)
y shape:
(1427,)
M_x shape:
(7402, 30, 1000)
M_y shape:
(7402,)
/content/drive/MyDrive/Extract_Driving_Events/s04_051130m_event_data.mat
1355
New data:
X shape:
(1355, 30

In [ ]:
print("Final Data:")
print("Master X: " + str(m_X.shape))
print("Master y: " + str(m_y.shape))

Final Data:
Master X: (18588, 30, 1000)
Master y: (18588,)


In [ ]:
np.save("master_data_X", m_X)

In [ ]:
np.save("master_data_Y", m_y)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/

with open('master_data_X.npy', 'wb') as f:
  np.save(f, m_X)


/content/drive/My Drive/Colab Notebooks


In [ ]:
%cd /MyDrive/Colab Notebooks/

with open('master_data_Y.npy', 'wb') as f:
  np.save(f, m_y)

[Errno 2] No such file or directory: '/MyDrive/Colab Notebooks/'
/content/drive/My Drive/Colab Notebooks


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import h5py

In [ ]:
dir_name = r'/content/drive/MyDrive/Extract_Driving_Events'

files_in_dir = [os.path.join(path, file) for path, dirs,
                files in os.walk(dir_name) for file in files]
print(files_in_dir)

['/content/drive/MyDrive/Extract_Driving_Events/s01_060227n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_1n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_2n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s02_050921m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s02_051115m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s04_051130m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_051120m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_060308n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_061019m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_061101n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s06_051119m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s09_060313n_event_data.mat']


In [ ]:
files_in_dir = files_in_dir[0:3]
print(len(files_in_dir))
print(files_in_dir)

3
['/content/drive/MyDrive/Extract_Driving_Events/s01_060227n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_1n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_2n_event_data.mat']


In [ ]:
m_X = np.empty((0, 30, 1000), float)
m_y = np.empty((0,), int)


for file_name in files_in_dir:
  f = h5py.File(file_name,'r')
  print(file_name)
  data = f.get('driver_baseline/eegData')
  data = np.array(data) # For converting to a NumPy array

  data = f.get('driver_baseline/eegData')
  data = np.array(data)
  baseline = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    baseline = np.concatenate((baseline,temp))
  #print(baseline.shape)

  data = f.get('driver_baseline/eegTime')
  data = np.array(data)
  baselineT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    baselineT = np.concatenate((baselineT,temp))
  #print(baselineT.shape)

  data = f.get('driver_correction/eegData')
  data = np.array(data)
  correction = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    correction = np.concatenate((correction,temp))
  #print(correction.shape)

  data = f.get('driver_correction/eegTime')
  data = np.array(data)
  correctionT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    correctionT = np.concatenate((correctionT,temp))
  #print(correctionT.shape)

  data = f.get('driver_disturbance/eegData')
  data = np.array(data)
  disturbance = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    disturbance = np.concatenate((disturbance,temp))
  #print(disturbance.shape)

  data = f.get('driver_disturbance/eegTime')
  data = np.array(data)
  disturbanceT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    disturbanceT = np.concatenate((disturbanceT,temp))
  #print(disturbanceT.shape)


  X = np.concatenate((baseline,correction,disturbance))
  #print("X success")
  #print(X.shape)
  Xt = np.concatenate((baselineT,correctionT,disturbanceT))
  #print("Xt success")
  #print(Xt.shape)
  y = [0 for i in range(len(baseline))] + [1 for i in range(len(correction))] + [2 for i in range(len(disturbance))]
  y = np.array(y)
  #print("y success")
  #print(y.shape)

  Xnew = np.ndarray((len(X)//1000+1,30,1000))
  ynew = []
  counter = 0
  size = 0
  i = 0
  while i < len(Xt)-1000:
    size = Xt[i+1]-Xt[i]

    if np.isclose(Xt[i+1000]-Xt[i],size*1000) :
      Xnew[counter] = np.transpose(X[i:i+1000])
      ynew.append(y[i])
      i = i+1000
      counter = counter+1
    else:
      i=i+1

  X = Xnew[:counter]
  y = ynew
  print(len(y))
  y = np.asarray(y)
  print("New data:")
  print("X shape:")
  print(X.shape)
  print("y shape:")
  print(y.shape)

  m_X = np.append(m_X, X, axis=0)
  print("M_x shape:")
  print(m_X.shape)
  m_y = np.append(m_y, y, axis=0)
  print("M_y shape:")
  print(m_y.shape)


/content/drive/MyDrive/Extract_Driving_Events/s01_060227n_event_data.mat
1403
New data:
X shape:
(1403, 30, 1000)
y shape:
(1403,)
M_x shape:
(1403, 30, 1000)
M_y shape:
(1403,)
/content/drive/MyDrive/Extract_Driving_Events/s01_060926_1n_event_data.mat
1717
New data:
X shape:
(1717, 30, 1000)
y shape:
(1717,)
M_x shape:
(3120, 30, 1000)
M_y shape:
(3120,)
/content/drive/MyDrive/Extract_Driving_Events/s01_060926_2n_event_data.mat
1456
New data:
X shape:
(1456, 30, 1000)
y shape:
(1456,)
M_x shape:
(4576, 30, 1000)
M_y shape:
(4576,)


In [ ]:
print("Final Data:")
print("Master X: " + str(m_X.shape))
print("Master y: " + str(m_y.shape))

Final Data:
Master X: (4576, 30, 1000)
Master y: (4576,)


In [ ]:
np.save("s01_data_X", m_X)

In [ ]:
np.save("s01_data_Y", m_y)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/

with open('s01_data_X.npy', 'wb') as f:
  np.save(f, m_X)


/content/drive/MyDrive/Colab Notebooks


In [ ]:
%cd /MyDrive/Colab Notebooks/

with open('s01_data_Y.npy', 'wb') as f:
  np.save(f, m_y)

[Errno 2] No such file or directory: '/MyDrive/Colab Notebooks/'
/content/drive/MyDrive/Colab Notebooks


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import h5py

In [ ]:
dir_name = r'/content/drive/MyDrive/Extract_Driving_Events'

files_in_dir = [os.path.join(path, file) for path, dirs,
                files in os.walk(dir_name) for file in files]
print(files_in_dir)

['/content/drive/MyDrive/Extract_Driving_Events/s01_060227n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_1n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s01_060926_2n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s02_050921m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s02_051115m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s04_051130m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_051120m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_060308n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_061019m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s05_061101n_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s06_051119m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s09_060313n_event_data.mat']


In [ ]:
files_in_dir = files_in_dir[3:5]
print(len(files_in_dir))
print(files_in_dir)

2
['/content/drive/MyDrive/Extract_Driving_Events/s02_050921m_event_data.mat', '/content/drive/MyDrive/Extract_Driving_Events/s02_051115m_event_data.mat']


In [ ]:
m_X = np.empty((0, 30, 1000), float)
m_y = np.empty((0,), int)


for file_name in files_in_dir:
  f = h5py.File(file_name,'r')
  print(file_name)
  data = f.get('driver_baseline/eegData')
  data = np.array(data) # For converting to a NumPy array

  data = f.get('driver_baseline/eegData')
  data = np.array(data)
  baseline = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    baseline = np.concatenate((baseline,temp))
  #print(baseline.shape)

  data = f.get('driver_baseline/eegTime')
  data = np.array(data)
  baselineT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    baselineT = np.concatenate((baselineT,temp))
  #print(baselineT.shape)

  data = f.get('driver_correction/eegData')
  data = np.array(data)
  correction = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    correction = np.concatenate((correction,temp))
  #print(correction.shape)

  data = f.get('driver_correction/eegTime')
  data = np.array(data)
  correctionT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    correctionT = np.concatenate((correctionT,temp))
  #print(correctionT.shape)

  data = f.get('driver_disturbance/eegData')
  data = np.array(data)
  disturbance = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    disturbance = np.concatenate((disturbance,temp))
  #print(disturbance.shape)

  data = f.get('driver_disturbance/eegTime')
  data = np.array(data)
  disturbanceT = np.zeros((0,f.get(data[0][0]).shape[1]))
  for i in range(data.shape[0]):
    dataset =  f.get(data[i][0])
    temp = np.zeros(dataset.shape)
    f.get(data[i][0]).read_direct(temp)
    disturbanceT = np.concatenate((disturbanceT,temp))
  #print(disturbanceT.shape)


  X = np.concatenate((baseline,correction,disturbance))
  #print("X success")
  #print(X.shape)
  Xt = np.concatenate((baselineT,correctionT,disturbanceT))
  #print("Xt success")
  #print(Xt.shape)
  y = [0 for i in range(len(baseline))] + [1 for i in range(len(correction))] + [2 for i in range(len(disturbance))]
  y = np.array(y)
  #print("y success")
  #print(y.shape)

  Xnew = np.ndarray((len(X)//1000+1,30,1000))
  ynew = []
  counter = 0
  size = 0
  i = 0
  while i < len(Xt)-1000:
    size = Xt[i+1]-Xt[i]

    if np.isclose(Xt[i+1000]-Xt[i],size*1000) :
      Xnew[counter] = np.transpose(X[i:i+1000])
      ynew.append(y[i])
      i = i+1000
      counter = counter+1
    else:
      i=i+1

  X = Xnew[:counter]
  y = ynew
  print(len(y))
  y = np.asarray(y)
  print("New data:")
  print("X shape:")
  print(X.shape)
  print("y shape:")
  print(y.shape)

  m_X = np.append(m_X, X, axis=0)
  print("M_x shape:")
  print(m_X.shape)
  m_y = np.append(m_y, y, axis=0)
  print("M_y shape:")
  print(m_y.shape)


/content/drive/MyDrive/Extract_Driving_Events/s02_050921m_event_data.mat
1399
New data:
X shape:
(1399, 30, 1000)
y shape:
(1399,)
M_x shape:
(1399, 30, 1000)
M_y shape:
(1399,)
/content/drive/MyDrive/Extract_Driving_Events/s02_051115m_event_data.mat
1427
New data:
X shape:
(1427, 30, 1000)
y shape:
(1427,)
M_x shape:
(2826, 30, 1000)
M_y shape:
(2826,)


In [ ]:
print("Final Data:")
print("Master X: " + str(m_X.shape))
print("Master y: " + str(m_y.shape))

Final Data:
Master X: (2826, 30, 1000)
Master y: (2826,)


In [ ]:
np.save("master_data_X", m_X)

In [ ]:
np.save("master_data_Y", m_y)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/

with open('s02_data_X.npy', 'wb') as f:
  np.save(f, m_X)


/content/drive/MyDrive/Colab Notebooks


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/

with open('s02_data_Y.npy', 'wb') as f:
  np.save(f, m_y)

/content/drive/MyDrive/Colab Notebooks
